## Load the libraries

In [1]:
from sqlalchemy import create_engine
import pandas as pd
import urllib.parse

## The connection with the database

In [2]:
# Create the connection to SQLAlchemy

conn_str = 'mssql+pyodbc:///?odbc_connect=' + \
           urllib.parse.quote_plus('DRIVER={SQL Server};SERVER=MSI\SQLEXPRESS;DATABASE=WideWorldImporters;Trusted_Connection=yes;')


In [3]:
# Create the engine to SQLAlchemy 

engine = create_engine(conn_str)

In [4]:
# I bring the query from sql server

sql_query = """
SELECT 
t1.OrderID as 'bill',
t2.StockItemID as 'productID',
t2.Description as 'product',
t2.Quantity as 'quantity',
t1.OrderDate as 'date',
t2.UnitPrice as 'price',
t1.CustomerID as 'customerID',
t3.ConfirmedReceivedBy as 'customer'

FROM sales.Orders as t1
	JOIN sales.OrderLines as t2 ON t1.OrderID = t2.OrderID
	JOIN sales.Invoices as t3 ON t1.OrderID = t3.OrderID
	WHERE t3.ConfirmedReceivedBy IS NOT NULL
"""

In [5]:
# Run the query and load the results into a Pandas DataFrame

df = pd.read_sql_query(sql_query, engine)

## Preprocessing the data

In [6]:
df.head()

,bill,productID,product,quantity,date,price,customerID,customer
0,1,67,Ride on toy sedan car (Black) 1/12 scale,10,2013-01-01,230.0,832,Aakriti Byrraju
1,2,50,Developer joke mug - old C developers never di...,9,2013-01-01,13.0,803,Bala Dixit
2,2,10,USB food flash drive - chocolate bar,9,2013-01-01,32.0,803,Bala Dixit
3,3,114,Superhero action jacket (Blue) XXL,3,2013-01-01,30.0,105,Sung-Hwan Hwang
4,4,206,Permanent marker black 5mm nib (Black) 5mm,96,2013-01-01,2.7,57,Aile Mae


In [7]:
df.shape

(227981, 8)

In [8]:
# Create customer matrix


customer_matrix = df.pivot_table(
    index='customerID', 
    columns='productID', 
    values='quantity',
    aggfunc='sum'
)

customer_matrix

productID,1,2,3,4,5,6,7,8,9,10,...,218,219,220,221,222,223,224,225,226,227
customerID,,,,,,,,,,,,,,,,,,,,,
1,3.0,22.0,340.0,NaN,NaN,5.0,6.0,15.0,8.0,26.0,...,80.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,48.0,NaN
2,11.0,NaN,170.0,NaN,16.0,9.0,12.0,24.0,NaN,NaN,...,60.0,80.0,NaN,12.0,NaN,NaN,144.0,NaN,96.0,NaN
3,4.0,5.0,NaN,5.0,13.0,14.0,13.0,8.0,23.0,25.0,...,100.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,11.0,130.0,24.0,NaN,4.0,7.0,NaN,9.0,9.0,...,110.0,60.0,144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,19.0,15.0,NaN,NaN,10.0,3.0,6.0,4.0,NaN,NaN,...,NaN,160.0,NaN,NaN,96.0,NaN,NaN,96.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,80.0,NaN,NaN,NaN,96.0,NaN,96.0,NaN,NaN
1058,NaN,NaN,NaN,NaN,3.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,120.0,NaN,NaN,NaN,288.0,NaN
1059,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Therefore, the pivot table customer_item_matrix will contain the total quantity of each product purchased by each customer, where the rows represent the customers, the columns represent the products, and the values ​​are the total quantities of products purchased. This pivot table is useful for analyzing customer purchasing patterns and product popularity.
If the customer didn't buy anything appear NaNs

In [9]:
# If the customer has purchased the product I give it a 1, otherwise 0

customer_matrix = customer_matrix.applymap(lambda x: 1 if x > 0 else 0)

C:\Users\34633\AppData\Local\Temp\ipykernel_15148\374284627.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  customer_matrix = customer_matrix.applymap(lambda x: 1 if x > 0 else 0)


In [10]:
customer_matrix.head()

productID,1,2,3,4,5,6,7,8,9,10,...,218,219,220,221,222,223,224,225,226,227
customerID,,,,,,,,,,,,,,,,,,,,,
1,1,1,1,0,0,1,1,1,1,1,...,1,1,0,0,0,0,0,0,1,0
2,1,0,1,0,1,1,1,1,0,0,...,1,1,0,1,0,0,1,0,1,0
3,1,1,0,1,1,1,1,1,1,1,...,1,1,0,0,0,0,0,0,0,0
4,1,1,1,1,0,1,1,0,1,1,...,1,1,1,0,0,0,0,0,0,0
5,1,1,0,0,1,1,1,1,0,0,...,0,1,0,0,1,0,0,1,0,0


## Similarity calculation

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
user_sim_matrix = pd.DataFrame(
    cosine_similarity(customer_matrix)
)

In [13]:
user_sim_matrix

,0,1,2,3,4,5,6,7,8,9,...,653,654,655,656,657,658,659,660,661,662
0,1.000000,0.817806,0.857907,0.820855,0.839614,0.832627,0.852624,0.829515,0.832667,0.826392,...,0.551819,0.481728,0.493972,0.479872,0.405737,0.407422,0.396818,0.357407,0.233285,0.310669
1,0.817806,1.000000,0.840630,0.748681,0.797753,0.807828,0.839894,0.804482,0.784142,0.801124,...,0.509207,0.426895,0.527186,0.485320,0.384187,0.440813,0.375742,0.325791,0.220353,0.250532
2,0.857907,0.840630,1.000000,0.811029,0.835207,0.844391,0.853482,0.846733,0.828296,0.832871,...,0.548922,0.479199,0.508929,0.495033,0.381790,0.445811,0.437410,0.328183,0.251398,0.309038
3,0.820855,0.748681,0.811029,1.000000,0.800118,0.810462,0.803828,0.772927,0.786140,0.792180,...,0.509378,0.474675,0.499316,0.428505,0.367840,0.405727,0.393482,0.317015,0.183406,0.368138
4,0.839614,0.797753,0.835207,0.800118,1.000000,0.818970,0.817792,0.798895,0.823917,0.806726,...,0.483746,0.506317,0.499917,0.485320,0.384187,0.430317,0.442050,0.354120,0.260417,0.347961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
658,0.407422,0.440813,0.445811,0.405727,0.430317,0.416328,0.443889,0.407046,0.424620,0.355850,...,0.269536,0.241113,0.271694,0.325036,0.211100,1.000000,0.185814,0.264628,0.187120,0.104010
659,0.396818,0.375742,0.437410,0.393482,0.442050,0.383575,0.402174,0.406618,0.402392,0.429793,...,0.267112,0.195292,0.268199,0.198142,0.222277,0.185814,1.000000,0.083592,0.157622,0.136896
660,0.357407,0.325791,0.328183,0.317015,0.354120,0.379267,0.348299,0.366234,0.315188,0.296629,...,0.213980,0.250313,0.206257,0.207791,0.085470,0.264628,0.083592,1.000000,0.151523,0.140372
661,0.233285,0.220353,0.251398,0.183406,0.260417,0.238384,0.256136,0.239046,0.263393,0.239713,...,0.151307,0.106199,0.226871,0.261209,0.161165,0.187120,0.157622,0.151523,1.000000,0.049629


In [14]:
# client_id becomes the index of the df

user_sim_matrix['customerID'] = customer_matrix.index
user_sim_matrix = user_sim_matrix.set_index('customerID')

In [15]:
# I have decided to use client number 5 though any of them can be used. It is a sample, 
# this one will later be modified by the selection of a particular client from powerBI.

customer_1 = 5

similarities = user_sim_matrix.loc[customer_1].sort_values(ascending=False) 

# I look for similar clients that customer_1 has

In [16]:
similarities = pd.DataFrame(similarities)
similarities

,5
4,1.000000
301,0.860075
358,0.857844
383,0.855631
239,0.854666
...,...
651,0.388629
657,0.384187
660,0.354120
662,0.347961


In [17]:
# reset the index

similarities = similarities.reset_index()
similarities

,index,5
0,4,1.000000
1,301,0.860075
2,358,0.857844
3,383,0.855631
4,239,0.854666
...,...,...
658,651,0.388629
659,657,0.384187
660,660,0.354120
661,662,0.347961


In [18]:
# columns are renamed

similarities.rename(columns={ similarities.columns[0]: "customers" }, inplace = True)
similarities.rename(columns={ similarities.columns[1]: "similarity" }, inplace = True)

In [19]:
similarities

#In this case CUSTOMERS ARE ORDERED ACCORDING TO THE GREATEST SIMILARITY THEY HAVE WITH THE CUSTOMER 240

,customers,similarity
0,4,1.000000
1,301,0.860075
2,358,0.857844
3,383,0.855631
4,239,0.854666
...,...,...
658,651,0.388629
659,657,0.384187
660,660,0.354120
661,662,0.347961


In [20]:
# obtain items purchased by customer_1

items_customer_1 = set(customer_matrix.loc[customer_1].iloc[
    customer_matrix.loc[customer_1].to_numpy().nonzero()
].index)

In [21]:
items_customer_1

{1,
 2,
 5,
 6,
 7,
 8,
 11,
 12,
 13,
 14,
 16,
 18,
 19,
 20,
 21,
 22,
 24,
 25,
 27,
 28,
 29,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 52,
 55,
 56,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 78,
 81,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 92,
 93,
 95,
 96,
 97,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 109,
 110,
 111,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 124,
 125,
 126,
 127,
 129,
 130,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 142,
 144,
 145,
 146,
 147,
 148,
 149,
 151,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 165,
 166,
 167,
 168,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 201,
 202,
 203,
 205,
 206,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 219,
 222,
 225}

In [22]:
# obtain items purchased by customer_2
# the same case as customer_1, the you can choose another customer in PowerBi

customer_2 = 999


items_customer_2 = set(customer_matrix.loc[customer_2].iloc[
    customer_matrix.loc[customer_2].to_numpy().nonzero()
].index)

In [23]:
items_customer_2

{1,
 2,
 3,
 4,
 5,
 6,
 9,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 22,
 25,
 26,
 27,
 30,
 31,
 34,
 36,
 37,
 38,
 41,
 42,
 44,
 45,
 47,
 48,
 50,
 52,
 53,
 54,
 55,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 68,
 71,
 72,
 73,
 74,
 75,
 76,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 89,
 90,
 91,
 92,
 94,
 95,
 96,
 97,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 114,
 115,
 118,
 120,
 122,
 123,
 124,
 126,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 148,
 149,
 150,
 151,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 161,
 162,
 163,
 164,
 165,
 166,
 168,
 169,
 171,
 172,
 173,
 174,
 177,
 178,
 179,
 180,
 182,
 183,
 184,
 186,
 188,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 212,
 213,
 215,
 217,
 218,
 222,
 223,
 224}

In [24]:
#I recommend to customer_2 based on what customer_1 has purchased

items_to_recommend_to_customer_2 = items_customer_1 - items_customer_2

In [25]:
items_to_recommend_to_customer_2

{7,
 8,
 21,
 24,
 28,
 29,
 32,
 33,
 35,
 39,
 40,
 43,
 46,
 49,
 56,
 67,
 69,
 70,
 78,
 87,
 88,
 93,
 111,
 113,
 116,
 117,
 119,
 121,
 125,
 127,
 136,
 146,
 147,
 160,
 167,
 175,
 176,
 181,
 185,
 187,
 211,
 214,
 216,
 219,
 225}

For now I only get the Product ID, I need more information about the product:

## Recommendations

In [26]:
# I need the name of the prodcut

result = df.loc[
    df['productID'].isin(items_to_recommend_to_customer_2), 
    ['productID', 'product']
].drop_duplicates().set_index('productID')

In [27]:
result

,product
productID,
67,Ride on toy sedan car (Black) 1/12 scale
121,Dinosaur battery-powered slippers (Green) XL
40,"Developer joke mug - (hip, hip, array) (White)"
39,Developer joke mug - inheritance is the OO way...
32,Developer joke mug - that's a hardware problem...
111,Superhero action jacket (Blue) M
21,DBA joke mug - you might be a DBA if (Black)
69,Ride on toy sedan car (Blue) 1/12 scale
70,Ride on toy sedan car (Green) 1/12 scale


In [28]:
# Reset the index

recommendations =  result.reset_index()
recommendations

,productID,product
0,67,Ride on toy sedan car (Black) 1/12 scale
1,121,Dinosaur battery-powered slippers (Green) XL
2,40,"Developer joke mug - (hip, hip, array) (White)"
3,39,Developer joke mug - inheritance is the OO way...
4,32,Developer joke mug - that's a hardware problem...
5,111,Superhero action jacket (Blue) M
6,21,DBA joke mug - you might be a DBA if (Black)
7,69,Ride on toy sedan car (Blue) 1/12 scale
8,70,Ride on toy sedan car (Green) 1/12 scale
9,35,Developer joke mug - fun was unexpected at thi...
